In [246]:
import pennylane as qml
from pennylane import numpy as np
dev1 = qml.device('default.qubit', wires=[0,1,2,3])
@qml.qnode(dev1)

#Defining the circuit function as given in the task

def my_quantum_function(theta):
  num_qubits = 4                   #### circuit has 4 qubit
  L = 9                                 #### no. of layers
  for _ in range(L):
    for i in range(num_qubits):
      qml.RX(theta[i+_*8], wires = i)
    for i in range(num_qubits):
      qml.RZ(theta[i+_*8+4], wires = i)
    for i in range(num_qubits):
      for j in range(i+1, num_qubits):
        qml.CZ(wires = [i,j])
  return qml.probs(wires=[0,1,2,3])

In [247]:
#defining the cost function as per the task

def cost(theta):

  # target_state is the required state

  target_state=[0.0280999 +0.05484259j, 0.14565419+0.00284204j, 0.34964817+0.20972156j,
                0.31944654+0.2474195j,  0.05660092+0.1189459j,  0.25226461+0.07934141j,
                0.27092873+0.06398543j, 0.18766363+0.23413237j, 0.21167778+0.06627202j,
                0.21332106+0.05929251j, 0.03998133+0.03564132j, 0.24241976+0.26835974j,
                0.03695061+0.0513821j,  0.31178641+0.17997049j, 0.03969748+0.0053141j,
                0.13355231+0.07578649j] 
 
  return np.sum(np.abs((my_quantum_function(theta)-np.abs(target_state)**2)))

In [248]:
L = 9 ## no of layers 

## randomly define the initial paramenters (theta) 

def initialize_theta(L):
    theta = 2*np.pi*np.random.rand(8*L)
    return theta

theta = np.array(initialize_theta(L))

In [249]:
dcost = qml.jacobian(my_quantum_function,argnum=0)

In [ ]:
#Training model

opt = qml.GradientDescentOptimizer(stepsize=0.04)

# set the number of steps
steps = 500
# set the initial parameter values
params = theta
c = []
p = []
i_m = []
for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)
    print("Cost after step {:5d}: {: .7f}".format(i, cost(params)))
    c.append(cost(params))
    p.append(params)
    i_m.append(i)
    


print("Optimized rotation angles: {}".format(params))


In [ ]:
#Plotting cost vs iteration 


from matplotlib import pyplot as plt

plt.style.use("seaborn")
plt.plot(i_m,c ,"b", label="gradient descent")
plt.ylabel("Cost function value")
plt.xlabel("Optimization steps")
plt.title('9 Layer Circuit')
plt.legend()
plt.show()